In [ ]:
import glob
import random
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm.notebook import trange, tqdm

In [ ]:
# for tile_heights folder for each city, calculate the rmse for summer, spring, winter 

files = glob.glob('data/tile_heights/*.csv')

for file in tqdm(files, total=len(files), leave=False):
    file = file.replace('\\','/')
    tiles = pd.read_csv(file)
    city = file.split('/')[-1].split('.')[0].split('-')[0]

    # create three columns for summer, spring, winter and add rmse values for each tile for three seasons
    tiles['summer'], tiles['spring'], tiles['winter'] = 0, 0, 0

    # iter tiles df
    for index, row in tqdm(tiles.iterrows(), total=tiles.shape[0], leave=False):
        # get i, j
        i = int(row['i'])
        j = int(row['j'])
        seasons = ['summer', 'spring', 'winter']
        for season in seasons:
            
            heights = np.empty([256, 256])
            img = cv2.imread('data/heights_new/%s/16/%d/%d.png'%(city,i,j)) 
            heights = img[:,:,0]

            img = cv2.imread('data/shadows_new/%s-%s/16/%d/%d.png'%(city,season,i,j)) 
            values_gan = img[:,:,0]
            values_gan[heights>0] = 0

            img = cv2.imread('data/shadows/%s-%s/16/%d/%d.png'%(city,season,i,j))
            values_true = img[:,:,0]
            values_true[heights>0] = 0

            target = values_true/255    #[:,128:-128,128:-128,:]
            target = target.astype(np.float32)
            prediction = values_gan/255     #[:,128:-128,128:-128,:]
            prediction = prediction.astype(np.float32)

            mse = np.mean((target-prediction) ** 2)
            rmse = np.sqrt(mse)
            tiles.at[index, season] = rmse # in place update of row[season]

    tiles.to_csv('data/tile_heights/%s-16.csv'%city, index=False)

In [ ]:
# combine all cities together in one dataframe
files = glob.glob('data/tile_heights/*.csv')
df = pd.DataFrame()
for file in tqdm(files, total=len(files), leave=False):
    file = file.replace('\\','/')
    city = file.split('/')[-1].split('.')[0].split('-')[0]
    tiles = pd.read_csv(file)
    tiles['city'] = city
    # without append
    df = pd.concat([df, tiles], ignore_index=True)

In [ ]:
df['avg'] = df[['summer', 'spring', 'winter']].mean(axis=1)
df

In [ ]:
# find quantiles of avg rmse
rmse_quantiles = df['avg'].quantile([0, 0.25, 0.5, 0.75, 1]).to_list()
rmse_iqr = rmse_quantiles[3] - rmse_quantiles[1]
print(rmse_quantiles)
print(rmse_iqr* 1.5)

In [ ]:
height_quantiles = df['height'].quantile([0, 0.25, 0.5, 0.75, 1]).to_list()
height_iqr = height_quantiles[3] - height_quantiles[1]
print(height_quantiles)

In [ ]:
df[(df['avg']<=rmse_quantiles[3]) & (df['height']<=height_quantiles[2]) & (df['city'] == 'chi')]

In [ ]:
colors = [(1, 1, 1), (0, 0, 0)]
cmap_g = LinearSegmentedColormap.from_list('colorscale', colors, N=256)

colors = [(1, 1, 1), (254/255,204/255,92/255), (253/255,141/255,60/255), (240/255,59/255,32/255), (189/255,0,38/255)]
cmap_r = LinearSegmentedColormap.from_list('colorscale', colors, N=100)

In [ ]:
test = df[(df['avg']>rmse_iqr* 1.5) & (df['city'] == 'chi') & (df['height']<=6.5)]
print(len(test))
# (df['avg']<=rmse_quantiles[3])
# (df['avg']>rmse_iqr* 1.5)
# & (df['height']<=height_quantiles[2])

# iter test. pick random 50
test = test.sample(n=50, random_state=1, replace=False)
for index, row in tqdm(test.iterrows(), total=test.shape[0], leave=False):
    # get i, j
    i = int(row['i'])
    j = int(row['j'])
    height = float(row['height'])
    avg = float(row['avg'])

    print(i,j, height, avg)
    city = 'chi'
    season = 'winter'

    heights = np.empty([256, 256])
    img = cv2.imread('data/heights_new/%s/16/%d/%d.png'%(city,i,j))
    heights = img[:,:,0]


    img = cv2.imread('data/shadows_new/%s-%s/16/%d/%d.png'%(city,season,i,j))
    values_gan = img[:,:,0]
    values_gan[heights>0] = 0

    img = cv2.imread('data/shadows/%s-%s/16/%d/%d.png'%(city,season,i,j))
    values_true = img[:,:,0]
    values_true[heights>0] = 0

    # subplot the three
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    ax1.imshow(heights/np.max(heights), cmap=cmap_g)
    ax2.imshow(values_gan, cmap=cmap_r)
    ax3.imshow(values_true, cmap=cmap_r)

    plt.show()


In [ ]:
# chi:
# small error: 16798 24361, 16798 24351, 16825 24384
# large error: 16814 24372, 16818 24354, 16815 24352

arr = [['chi', 16798, 24361], ['chi', 16798, 24351], ['chi', 16825, 24384], ['chi', 16814, 24372], ['chi', 16818, 24354], ['chi', 16815, 24352]]

In [ ]:
# tok:
# small error 58195 25809, 58233 25819, 58218 25808
# large error 58206 25810, 58212 25805, 58198 25805, 58207 25807, 58204 25820

# extend arr with tok cases
arr.extend([['tok', 58195, 25809], ['tok', 58233, 25819], ['tok', 58218, 25808], ['tok', 58206, 25810], ['tok', 58212, 25805], ['tok', 58198, 25805], ['tok', 58207, 25807], ['tok', 58204, 25820]])

In [ ]:
# bue:
# small error: 22140 39495, 22138 39491, 22144 39496
# large error: 22141 39488, 22128 39503, 22143 39491, 22141 39489

arr.extend([['bue', 22140, 39495], ['bue', 22138, 39491], ['bue', 22144, 39496], ['bue', 22141, 39488], ['bue', 22128, 39503], ['bue', 22143, 39491], ['bue', 22141, 39489]])

In [ ]:
# Folders like chi_16798_24351/summer, chi_16798_24351/spring, chi_16798_24351/winter
# And then inside each folder, put heights, values_gan, values_true

seasons = ['summer', 'spring', 'winter']
for city, i, j in tqdm(arr, total=len(arr), leave=False):
    for season in seasons:
        heights = np.empty([256, 256])
        img = cv2.imread('data/heights_new/%s/16/%d/%d.png'%(city,i,j))
        heights = img[:,:,0]


        img = cv2.imread('data/shadows_new/%s-%s/16/%d/%d.png'%(city,season,i,j))
        values_gan = img[:,:,0]
        values_gan[heights>0] = 0

        img = cv2.imread('data/shadows/%s-%s/16/%d/%d.png'%(city,season,i,j))
        values_true = img[:,:,0]
        values_true[heights>0] = 0

        # save to folder. plt axis off
        folder = 'data/plots/fig4/%s_%d_%d/%s'%(city,i,j,season)
        if not os.path.exists(folder):
            os.makedirs(folder)

        plt.imshow(heights, cmap=cmap_g)
        plt.axis('off')
        plt.savefig('%s/heights.png'%folder, bbox_inches='tight', pad_inches=0)
        plt.close()

        plt.imshow(values_gan, cmap=cmap_r)
        plt.axis('off')
        plt.savefig('%s/values_gan.png'%folder, bbox_inches='tight', pad_inches=0)
        plt.close()

        plt.imshow(values_true, cmap=cmap_r)
        plt.axis('off')
        plt.savefig('%s/values_true.png'%folder, bbox_inches='tight', pad_inches=0)
        plt.close()
